In [1]:
!pip install Pillow opencv-python numpy transformers torch moviepy scikit-image dlib face_recognition accelerate bitsandbytes timm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.1/100.1 MB 11.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 69.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 60.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 35.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 63.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 10.9 MB/s eta 0:00:00
  

# For Gemma model access (if you haven't done this already):
# 1. Go to https://huggingface.co/google/gemma-2b-it and accept its terms.
# 2. Go to https://huggingface.co/settings/tokens to generate a new access token (e.g., 'gemma_access').
# 3. In Colab, click the '🔑 Secrets' icon on the left sidebar.
# 4. Add a new secret named 'HF_TOKEN' and paste your Hugging Face token as its value.
# 5. Ensure the toggle next to 'HF_TOKEN' is ON for this notebook.

In [2]:
import os
import glob
from PIL import Image, ImageEnhance, ImageFilter
import cv2
import numpy as np
from moviepy.editor import VideoFileClip
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM, AutoFeatureExtractor, AutoModel # Import AutoFeatureExtractor, AutoModel
import torch
import face_recognition # For face detection (AI)
import textwrap # For neatly wrapping LLM output

  if event.key is 'enter':



# --- Configuration ---

In [3]:
OUTPUT_DIR = "processed_social_media_photos"
MAX_PHOTOS_TO_SELECT = 10 # Maximum photos to process and consider for the album vibe
TARGET_ASPECT_RATIOS = [(4,5)] # Preferred aspect ratios for social media (e.g., Instagram square, portrait)

In [4]:
# --- Top Shot dependencies ---
!pip install --quiet timm tflite-runtime opencv-python-headless==4.10.0.82

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 82.7 MB/s eta 0:00:00


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
%run '/content/drive/MyDrive/best_photo_selector.ipynb'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.8/46.8 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 8.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 38.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 30.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.6/108.6 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 83.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 70.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that

  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)



open_clip_pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

Cloning into 'yolo-face'...
remote: Enumerating objects: 331, done.
remote: Counting objects: 100% (331/331), done.
remote: Compressing objects: 100% (281/281), done.
remote: Total 331 (delta 48), reused 227 (delta 43), pack-reused 0 (from 0)
Receiving objects: 100% (331/331), 35.83 MiB | 39.93 MiB/s, done.
Resolving deltas: 100% (48/48), done.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 22.9 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


Scoring:   0%|          | 0/8 [00:00<?, ?it/s]

...


0: 640x480 1 face, 9.9ms
Speed: 4.3ms preprocess, 9.9ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 480)


Scoring:  88%|████████▊ | 7/8 [00:03<00:00,  2.52it/s]


0: 640x480 1 face, 11.8ms
Speed: 4.2ms preprocess, 11.8ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 480)


Scoring: 100%|██████████| 8/8 [00:04<00:00,  1.85it/s]

[(np.float64(0.6956167736529094), PosixPath('/content/drive/My Drive/bursts/IMG_1.jpg'), {'sharp': np.float64(0.8139828824678464), 'expo': np.float32(0.8625286), 'aes': np.float64(0.23490152756373087), 'center': np.float64(0.8612610364749685), 'smile': np.float64(0.8784277439117432)}), (np.float64(0.6695943844928872), PosixPath('/content/drive/My Drive/bursts/IMG_3.jpg'), {'sharp': np.float64(0.6733251802790946), 'expo': np.float32(0.87269425), 'aes': np.float64(0.23714296023050943), 'center': np.float64(0.8883028648532112), 'smile': np.float64(0.8894175887107849)}), (np.float64(0.6636774626561901), PosixPath('/content/drive/My Drive/bursts/IMG_5.jpg'), {'sharp': np.float64(0.7672163629774306), 'expo': np.float32(0.8613354), 'aes': np.float64(0.18275392055511472), 'center': np.float64(0.8341800675431328), 'smile': np.float64(0.8676571249961853)}), (np.float64(0.654238358377704), PosixPath('/content/drive/My Drive/bursts/IMG_2.jpg'), {'sharp': np.float64(0.6235242556855715), 'expo': np.

# --- AI Models Configuration ---

In [8]:
BLIP_MODEL_NAME = "Salesforce/blip-image-captioning-base" # For individual image descriptions (Vision-Encoder-Decoder AI)
GEMMA_LLM_MODEL = "google/gemma-2b-it" # For album caption generation (Large Language Model AI)
AESTHETIC_MODEL_NAME = "HuggingFaceH4/aesthetic-predictor-v2" # For aesthetic scoring (Dedicated AI model)

# --- AI Aesthetic Scorer (Perfect AI Matrix Integration) ---

In [9]:
class AestheticScorer:
    def __init__(self):
        print("\nLoading AI Aesthetic Predictor Model...")
        self.processor = None
        self.model = None
        try:
            # Load feature extractor (for preprocessing images) and the model
            self.processor = AutoFeatureExtractor.from_pretrained(AESTHETIC_MODEL_NAME)
            self.model = AutoModel.from_pretrained(AESTHETIC_MODEL_NAME)
            # Move model to GPU if available
            self.device = "cuda" if torch.cuda.is_available() else "cpu"
            self.model.to(self.device)
            self.model.eval() # Set model to evaluation mode
            print(f"Loaded AI Aesthetic Predictor: {AESTHETIC_MODEL_NAME} on {self.device}.")
        except Exception as e:
            print(f"ERROR: Could not load AI Aesthetic Predictor model '{AESTHETIC_MODEL_NAME}'.")
            print("Please ensure you have all necessary libraries installed (e.g., 'timm').")
            print(f"Details: {e}")
            self.processor = None
            self.model = None
            print("Aesthetic scoring will be skipped or may cause errors.")

    def predict_aesthetic_score(self, pil_image):
        """
        Predicts an aesthetic score for an image using a pre-trained AI model.
        The score typically ranges from 1 to 10.
        """
        if self.processor is None or self.model is None:
            # Fallback to a very basic heuristic or return a default score if model loading failed
            print("Warning: Aesthetic predictor model not loaded. Using fallback score.")
            return 5.0 # Neutral score if AI model failed to load

        try:
            # Preprocess the image using the model's feature extractor
            inputs = self.processor(images=pil_image, return_tensors="pt")
            inputs = {k: v.to(self.device) for k, v in inputs.items()} # Move inputs to device

            # Perform inference
            with torch.no_grad(): # Disable gradient calculation for inference
                outputs = self.model(**inputs)

            # The model's output is typically a single logit.
            # Convert logit to a score, often scaled to 1-10.
            # For HuggingFaceH4/aesthetic-predictor-v2, a common scaling is to interpret
            # the single logit output as a direct score (e.g., 1-10 or 0-1).
            # If the model outputs logits, a sigmoid or other activation might be needed.
            # Based on common usage, a direct interpretation or clipping is often applied.
            # Let's assume the output is a single float representing the score.
            score = outputs.logits.squeeze().item() # Get the scalar value

            # A common practice is to scale it if the model outputs raw logits or a different range.
            # For this model, scores typically range from 1 to 10. We can clip for safety.
            aesthetic_score = max(1.0, min(10.0, score)) # Ensure score is within 1 to 10 range

            return aesthetic_score

        except Exception as e:
            print(f"Error during aesthetic prediction for image: {e}")
            return 5.0 # Return a neutral score on prediction error


# --- Utility Functions ---

In [10]:
def get_image_paths(album_path):
    """Gathers all image and potential Live Photo paths."""
    image_extensions = ['jpg', 'jpeg', 'png', 'webp']
    all_files = os.listdir(album_path)

    photos = []
    live_photos_map = {}

    for filename in all_files:
        name, ext = os.path.splitext(filename)
        ext = ext.lower().lstrip('.')
        full_path = os.path.join(album_path, filename)

        if ext in image_extensions:
            # Check if there's a corresponding .mov for Live Photos
            if os.path.exists(os.path.join(album_path, name + '.mov')):
                if name not in live_photos_map:
                    live_photos_map[name] = {}
                live_photos_map[name]['jpg'] = full_path
            else:
                photos.append({'type': 'still', 'path': full_path})
        elif ext == 'mov':
            if os.path.exists(os.path.join(album_path, name + '.jpg')):
                if name not in live_photos_map:
                    live_photos_map[name] = {}
                live_photos_map[name]['mov'] = full_path

    for name, files in live_photos_map.items():
        if 'jpg' in files and 'mov' in files:
            photos.append({'type': 'live', 'jpg_path': files['jpg'], 'mov_path': files['mov']})

    return photos

# --- Live Photo "Best Shot" Selection (AI-driven face analysis) ---

In [11]:
def detect_blur(image_np):
    """Calculates a blur score for an image using Laplacian variance."""
    gray = cv2.cvtColor(image_np, cv2.COLOR_BGR2GRAY)
    return cv2.Laplacian(gray, cv2.CV_64F).var()

def analyze_faces_for_quality(image_np):
    """
    AI-driven Face Analysis:
    Detects faces and evaluates quality factors like open eyes, smiles.
    This is a conceptual placeholder; real implementations use ML models.
    """
    face_locations = face_recognition.face_locations(image_np)
    if not face_locations:
        return 0, 0 # No faces, no face quality score

    total_face_score = 0

    for face_location in face_locations:
        # A more advanced AI would detect specific expressions, eye closure, etc.
        face_score = 100 # Base score for just having a face

        # Dummy conditions for demonstration, replace with actual ML inference for expressions:
        # e.g., using facial landmarks and a classifier for smiles/open eyes

        total_face_score += face_score

    return len(face_locations), total_face_score

def select_best_live_photo_frame(mov_path):
    """
    Extracts frames from a Live Photo video and selects the "best" one.
    "Best" is determined by an AI-informed heuristic score based on blur and face quality.
    """
    print(f"Processing Live Photo: {mov_path}")
    best_frame = None
    best_score = -1

    try:
        clip = VideoFileClip(mov_path)

        # Process a limited number of frames to avoid excessive computation
        frame_interval = max(1, int(clip.duration / 10)) # Get approx 10 frames

        for i, frame_time in enumerate(np.arange(0, clip.duration, frame_interval)):
            if i >= 20: # Limit total frames processed to 20
                break
            frame_np = clip.get_frame(frame_time) # RGB array

            # Convert to BGR for OpenCV
            frame_bgr = cv2.cvtColor(frame_np, cv2.COLOR_RGB2BGR)

            blur_score = detect_blur(frame_bgr)
            num_faces, face_quality_score = analyze_faces_for_quality(frame_bgr)

            # AI-informed scoring heuristic for best frame:
            current_frame_score = blur_score * 0.5 + face_quality_score * 1.0

            if current_frame_score > best_score:
                best_score = current_frame_score
                best_frame = frame_np

        if best_frame is not None:
            return Image.fromarray(best_frame)

    except Exception as e:
        print(f"Error processing Live Photo {mov_path}: {e}")
    return None

# --- Image Quality & Engagement Scoring (AI-driven selection) ---

In [12]:
def score_photo_engagement(pil_image, aesthetic_scorer):
    """
    Scores a photo based on aesthetic and engagement potential.
    Leverages AI for aesthetic prediction.
    """
    img_np = np.array(pil_image.convert('RGB'))
    img_bgr = cv2.cvtColor(img_np, cv2.COLOR_RGB2BGR)

    # 1. AI Aesthetic Score (Most important component for "best shot")
    ai_aesthetic_score = aesthetic_scorer.predict_aesthetic_score(pil_image)

    # 2. Face Presence & Quality Score (AI-informed)
    num_faces, face_quality_score = analyze_faces_for_quality(img_bgr)

    # 3. Basic Photography Metrics (supporting AI)
    blur_score = detect_blur(img_bgr)
    gray_image = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2GRAY)
    brightness_mean = np.mean(gray_image)
    exposure_deviation = abs(brightness_mean - 128)

    # Combine scores with weights. Tune these weights!
    # The AI aesthetic score is now a direct, powerful component.
    total_score = (ai_aesthetic_score * 100) + \
                  (face_quality_score * 1.5) + \
                  (blur_score * 0.1) + \
                  (255 - exposure_deviation)

    return total_score

# --- Social Media-Ready Editing ---

In [13]:
def apply_social_media_edits(pil_image, preferred_aspect_ratio=(4,5)):
    """Applies common social media edits: crop, enhance, sharpen."""
    img_width, img_height = pil_image.size

    # Smart Cropping (AI for object-aware cropping is a future enhancement)
    target_w, target_h = preferred_aspect_ratio

    potential_h = int(img_width * (target_h / target_w))
    if potential_h <= img_height:
        crop_width = img_width
        crop_height = potential_h
        left = 0
        top = (img_height - crop_height) // 2
    else:
        crop_height = img_height
        crop_width = int(img_height * (target_w / target_h))
        left = (img_width - crop_width) // 2
        top = 0

    right = left + crop_width
    bottom = top + crop_height

    pil_image = pil_image.crop((left, top, right, bottom))

    # Basic Enhancements (can be AI-stylized in future)
    enhancer = ImageEnhance.Contrast(pil_image)
    pil_image = enhancer.enhance(1.1)

    enhancer = ImageEnhance.Color(pil_image)
    pil_image = enhancer.enhance(1.1)

    pil_image = pil_image.filter(ImageFilter.SHARPEN)

    return pil_image

# --- Album-Level Vibe Caption Generation (Fully AI-powered) ---

In [14]:
class AlbumVibeCaptionGenerator:
    def __init__(self):
        print("\nLoading AI models for captioning and vibe generation...")
        # BLIP for individual image descriptions (Vision-Encoder-Decoder)
        self.blip_captioner = pipeline("image-to-text", model=BLIP_MODEL_NAME, device=0 if torch.cuda.is_available() else -1)
        print(f"Loaded BLIP model: {BLIP_MODEL_NAME}")

        # Gemma for LLM-based album caption summarization and vibe generation
        # Ensure you have accepted the model terms on Hugging Face and set up HF_TOKEN secret in Colab
        try:
            self.llm_tokenizer = AutoTokenizer.from_pretrained(GEMMA_LLM_MODEL)
            self.llm_model = AutoModelForCausalLM.from_pretrained(
                GEMMA_LLM_MODEL,
                torch_dtype=torch.float16, # Use float16 for memory efficiency
                device_map="auto", # Automatically map to GPU if available
                token=os.environ.get("HF_TOKEN") # Use HF_TOKEN from Colab Secrets
            )
            print(f"Loaded LLM model: {GEMMA_LLM_MODEL}")
        except Exception as e:
            print(f"ERROR: Could not load LLM model '{GEMMA_LLM_MODEL}'.")
            print("Please ensure you have accepted its terms on Hugging Face and set up your HF_TOKEN secret in Colab.")
            print(f"Details: {e}")
            self.llm_tokenizer = None
            self.llm_model = None

    def generate_individual_captions(self, pil_images):
        """Generates captions for each image using the BLIP model."""
        individual_captions = []
        if self.blip_captioner:
            print("Generating individual image descriptions...")
            for i, img in enumerate(pil_images):
                try:
                    results = self.blip_captioner(img)
                    if results:
                        caption = results[0]['generated_text']
                        individual_captions.append(f"Image {i+1}: {caption}")
                except Exception as e:
                    print(f"  Error captioning image {i+1}: {e}")
                    individual_captions.append(f"Image {i+1}: Could not describe this image.")
        return individual_captions

    def generate_album_vibe_caption(self, individual_captions):
        """
        Generates a single, vibe-fitting caption for the entire album using an LLM.
        """
        if not self.llm_model:
            return "Could not generate album caption due to LLM loading error. Check Colab setup."

        # Craft a prompt for the LLM based on the individual captions
        context = "\n".join(individual_captions)

        prompt = textwrap.dedent(f"""
        You are a highly creative and engaging social media caption generator for influencers.
        You have analyzed a collection of photos from an album.
        Below are individual descriptions of the key photos in the album.

        Based on these descriptions, identify the overall theme, mood, and "vibe" of the entire album.
        Then, generate a single, compelling, and short social media caption (max 3 sentences) that captures this vibe.
        Include relevant emojis and 3-5 popular hashtags to maximize engagement.
        Make it sound authentic and inspiring for social media influencers.

        Individual photo descriptions:
        ---
        {context}
        ---

        Album Vibe Caption:
        """)

        # Tokenize and generate with the LLM

        inputs = self.llm_tokenizer(prompt, return_tensors="pt").to(self.llm_model.device)

        outputs = self.llm_model.generate(
            **inputs,
            max_new_tokens=100, # Adjust for desired length of the album caption
            temperature=0.8,
            do_sample=True,
            pad_token_id=self.llm_tokenizer.eos_token_id
        )

        generated_text = self.llm_tokenizer.decode(outputs[0], skip_special_tokens=True)

        response_start = generated_text.find("Album Vibe Caption:")
        if response_start != -1:
            generated_caption = generated_text[response_start + len("Album Vibe Caption:"):].strip()
        else:
            generated_caption = generated_text

        generated_caption = generated_caption.split("Individual photo descriptions:")[0].strip()

        if not any(tag.startswith('#') for tag in generated_caption.split()):
            generated_caption += "\n#AlbumVibes #InfluencerLife #ContentCreator"

        return generated_caption

# --- Main Processing Logic ---

In [15]:
def process_album(album_path):
    if not os.path.exists(album_path):
        print(f"Error: Album path '{album_path}' does not exist.")
        return

    os.makedirs(OUTPUT_DIR, exist_ok=True)

    # Initialize AI models
    album_caption_generator = AlbumVibeCaptionGenerator() # Handles both individual and album-level AI captioning
    aesthetic_scorer = AestheticScorer() # AI aesthetic scoring (now a real model)

    all_photos_info = [] # To store loaded images and their scores

    # 1. Gather all photos and process Live Photos (AI-informed best shot)
    photo_paths = get_image_paths(album_path)

    if not photo_paths:
        print(f"No image files found in '{album_path}'. Please check the directory and file types.")
        return

    for i, photo_entry in enumerate(photo_paths):
        print(f"[{i+1}/{len(photo_paths)}] Processing {photo_entry['type']} photo...")
        pil_image = None
        original_path = ""

        if photo_entry['type'] == 'live':
            original_path = photo_entry['jpg_path']
            best_frame = select_best_live_photo_frame(photo_entry['mov_path'])
            if best_frame:
                pil_image = best_frame
                print(f"Selected best frame from Live Photo: {os.path.basename(original_path)}")
            else:
                print(f"Could not process Live Photo: {os.path.basename(original_path)}. Skipping.")
                continue
        else: # Still photo
            original_path = photo_entry['path']
            try:
                pil_image = Image.open(original_path).convert('RGB')
                print(f"Successfully loaded still photo: {os.path.basename(original_path)}")
            except Exception as e:
                print(f"Error opening still photo {original_path}: {e}. Skipping.")
                continue

        if pil_image:
            # Here's where the *real* AI aesthetic score is used
            score = score_photo_engagement(pil_image, aesthetic_scorer)
            all_photos_info.append({
                'original_path': original_path,
                'pil_image': pil_image,
                'score': score
            })
            print(f"  -> Scored {os.path.basename(original_path)}: {score:.2f} (AI Aesthetic Score)")

    # 2. Select top N photos based on AI-driven score
    all_photos_info.sort(key=lambda x: x['score'], reverse=True)
    selected_photos_info = all_photos_info[:MAX_PHOTOS_TO_SELECT]

    if not selected_photos_info:
        print("No suitable photos found or selected based on AI aesthetic and engagement scoring. Try adding more photos or adjusting MAX_PHOTOS_TO_SELECT.")
        return

    print(f"\nSelected {len(selected_photos_info)} best photos based on AI aesthetic and engagement scoring.")

    # Store individual processed images for album captioning
    processed_pil_images_for_captioning = []

    # 3. Process selected photos: edit, save
    print("\nApplying social media edits and saving selected photos...")
    for idx, photo_data in enumerate(selected_photos_info):
        original_img_path = photo_data['original_path']
        edited_pil_image = photo_data['pil_image'] # Start with original or best frame

        # Apply edits (you can customize the aspect ratio here)
        edited_pil_image = apply_social_media_edits(edited_pil_image, preferred_aspect_ratio=TARGET_ASPECT_RATIOS[0])

        # Save edited image
        output_filename = f"post_{idx+1}_{os.path.basename(original_img_path)}"
        output_filepath = os.path.join(OUTPUT_DIR, output_filename)
        edited_pil_image.save(output_filepath)
        processed_pil_images_for_captioning.append(edited_pil_image) # Add to list for album captioning

        print(f"  Saved edited photo: {output_filepath}")

    # 4. Generate a single, vibe-fitting caption for the entire album (Fully AI-powered)
    print("\nGenerating album-level caption using LLM...")
    individual_blip_captions = album_caption_generator.generate_individual_captions(processed_pil_images_for_captioning)
    album_vibe_caption = album_caption_generator.generate_album_vibe_caption(individual_blip_captions)

    # Write the album caption to a file
    album_caption_filepath = os.path.join(OUTPUT_DIR, "album_vibe_caption.txt")
    with open(album_caption_filepath, 'w', encoding='utf-8') as f_album_caption:
        f_album_caption.write("--- Album Vibe Caption for All Selected Photos ---\n\n")
        f_album_caption.write(textwrap.fill(album_vibe_caption, width=80)) # Wrap for readability
        f_album_caption.write("\n\n--- Individual Photo Descriptions (for reference) ---\n\n")
        for cap in individual_blip_captions:
            f_album_caption.write(textwrap.fill(cap, width=80) + "\n")

    print(f"\nAll processed photos saved to: {os.path.abspath(OUTPUT_DIR)}")
    print(f"Album vibe caption and individual descriptions saved to: {os.path.abspath(album_caption_filepath)}")
    print("\nProcessing Complete!")

# --- Colab Specific Execution Block ---

In [16]:
if __name__ == "__main__":
    from google.colab import drive
    import os

    # IMPORTANT: Please accept Gemma's terms and set up your HF_TOKEN in Colab Secrets
    # (See instructions at the top of this code block)

    print("Mounting Google Drive...")
    drive.mount('/content/drive')

    # Define the path to your photo album in Google Drive.
    # *** IMPORTANT: Change this to the ACTUAL path where you uploaded your photos! ***
    # Example: If your folder is named 'my_influencer_photos' directly in MyDrive:
    album_directory = "/content/drive/MyDrive/burstoutput"

    # --- Colab Setup Instructions ---
    print("\n--- Colab Setup Reminder ---")
    print("1. Ensure you have run the `!pip install ...` cell at the top.")
    print("2. Ensure you have accepted Gemma's terms on Hugging Face AND the terms for 'HuggingFaceH4/aesthetic-predictor-v2' ")
    print("   if prompted, and added your HF_TOKEN to Colab Secrets.")
    print("3. Upload your photo album folder to your Google Drive, and update 'album_directory' above to its exact path.")
    print(f"   Currently set to: {album_directory}")
    print("\nStarting AI photo processing...")
    # -------------------------------

    process_album(album_directory)

Mounting Google Drive...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).

--- Colab Setup Reminder ---
1. Ensure you have run the `!pip install ...` cell at the top.
2. Ensure you have accepted Gemma's terms on Hugging Face AND the terms for 'HuggingFaceH4/aesthetic-predictor-v2' 
   if prompted, and added your HF_TOKEN to Colab Secrets.
3. Upload your photo album folder to your Google Drive, and update 'album_directory' above to its exact path.
   Currently set to: /content/drive/MyDrive/burstoutput

Starting AI photo processing...

Loading AI models for captioning and vibe generation...


config.json:   0%|          | 0.00/4.56k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/990M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/506 [00:00<?, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

Loaded LLM model: google/gemma-2b-it

Loading AI Aesthetic Predictor Model...
ERROR: Could not load AI Aesthetic Predictor model 'HuggingFaceH4/aesthetic-predictor-v2'.
Please ensure you have all necessary libraries installed (e.g., 'timm').
Details: HuggingFaceH4/aesthetic-predictor-v2 is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`
Aesthetic scoring will be skipped or may cause errors.
[1/5] Processing still photo...
Successfully loaded still photo: IMG_1.jpg
  -> Scored IMG_1.jpg: 890.74 (AI Aesthetic Score)
[2/5] Processing still photo...
Successfully loaded still photo: IMG_3.jpg
  -> Scored IMG_3.jpg: 886.57 (AI Aesthetic Score)
[3/5] Processing still photo...
Successfully loaded still photo: IMG_5.jpg
  -> Scored IMG_5.jpg: 743.70 (AI Aesthetic Score)
[4/5